In [1]:
import codecs
import os
import pandas as pd
import re
from os import listdir
from os.path import isfile, join
from utils import printProgressBar

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
context_with_header = pd.read_csv('context_with_header.csv', header=0, index_col=0)
col_names = context_with_header.columns
family_ids = context_with_header.index

families = [f for f in listdir('families') if isfile(join('families', f))]
families_dict = dict()
for f in families:
    elements = f.replace('.txt', '').split()
    families_dict[elements[0]] = elements[1].split('_')[0]

In [3]:
# filter by family, simplified
families = [f for f in listdir('families') if isfile(join('families', f))]
group_prev = ''
counter = 0
for file_name in families:
    family_id = file_name.split()[0]
    group_id = family_id.split('-')[0]
    if group_id == group_prev:
        continue
    group_prev = group_id
    full_dot = 'families_full.dot'
    simplified_dot = 'families_simplified.dot'
    directory = 'posets'
    out_directory = 'posets_all'

    #family_number = int(family_id.strip('F'))
    object_id = family_ids.get_loc(family_id)
    out_file_name = 'posets_' + family_id + '_simplified' + '.dot'
    out_file = codecs.open(join(out_directory, out_file_name), 'w')

    object_concept = ''
    with codecs.open(join(directory, simplified_dot), 'r', encoding='utf-8') as f:
        for line in f:
            if ('Object ' + str(object_id) + '\\n') in line:
                object_concept = line.split()[0]
                break

    attributes_of_object_concept = list()
    with codecs.open(join(directory, full_dot), 'r', encoding='utf-8') as f:
        for line in f:
            if object_concept in line:
                attributes_of_object_concept = line.split('|')[1].split('\\n')[0:-1]
                break

    selected_concepts = list()    
    with codecs.open(join(directory, full_dot), 'r', encoding='utf-8') as f:
        for line in f:
            if '->' in line:
                break
            if 'graph' in line or 'rankdir' in line:
                continue
            concept_id = line.split()[0]
            if ('Object ' + str(object_id) + '\\n') in line:
                selected_concepts.append(concept_id)
            else:
                selected = True
                for att in attributes_of_object_concept:
                    if att not in line:
                        selected = False
                        break
                if selected:
                    selected_concepts.append(concept_id)

    with codecs.open(join(directory, simplified_dot), 'r', encoding='utf-8') as f:
        for line in f:
            if 'graph' in line or 'rankdir' in line or line == '}':
                out_file.write(line)
                continue
            if '->' in line:
                elements = line.split()
                conc1 = elements[0]
                conc2 = elements[2]
                if conc1 in selected_concepts and conc2 in selected_concepts:
                    out_file.write(line)
                continue
            vertex_id = line.split()[0]
            concept_id = re.search('<(.*)>', line).group(1)
            if vertex_id in selected_concepts:
                if 'E: 13179' in line: # assumption : number of families = 13179, top concept has empty intent
                    elements = line.split('|')
                    elements[2] = '||all families}"];\n'
                    to_be_written = ''.join(elements)
                    to_be_written = re.sub('\(.*\)', concept_id, to_be_written)
                    out_file.write(to_be_written)
                else:
                    #line = line.replace('Attribute', 'Graph').replace('Object ', 'F')
                    to_be_written = line.split('|')[0] + '|'
                    attribute_string = line.split('|')[1]
                    if 'Attribute' not in attribute_string: # empty intent
                        to_be_written += '|'
                    else:
                        attributes = attribute_string.split('\\n')
                        for attribute in attributes:
                            if attribute == '':
                                continue
                            to_be_written += col_names[int(attribute.split()[1])] + '\\n'
                    to_be_written += '|'
                    object_string = line.split('|')[2]
                    if 'Object' not in object_string: # empty extent
                        to_be_written += '|'
                    else:
                        objects = object_string.split('\\n')
                        for obj in objects:
                            if obj == '' or '}' in obj:
                                continue
                            to_be_written += families_dict[family_ids[int(obj.split()[1])]] + '\\n'
                    to_be_written += '}"];\n'
                    to_be_written = re.sub('\(.*\)', concept_id, to_be_written)
                    out_file.write(to_be_written)
    out_file.close()
    #os.system('dot -Tpdf ' + join(out_directory, out_file_name) + ' -o ' + join(out_directory, out_file_name).replace('.dot', '.pdf'))
    counter += 1
    printProgressBar(counter, 3903, prefix = 'Progress:', suffix = 'Complete', length = 50)

KeyboardInterrupt: 

In [ ]:
# filter by family, full
family_id = 'F01942'
full_dot = 'families_full_s100_l3.dot'
simplified_dot = 'families_simplified_s100_l3.dot'
directory = 'posets'

family_number = int(family_id.strip('F'))
out_file_name = 'posets_' + family_id + '_full' + '.dot'
out_file = codecs.open(join(directory, out_file_name), 'w')

object_concept = ''
with codecs.open(join(directory, simplified_dot), 'r', encoding='utf-8') as f:
    for line in f:
        if ('Object ' + str(family_number)) in line:
            object_concept = line.split()[0]
            break
            
attributes_of_object_concept = list()
with codecs.open(join(directory, full_dot), 'r', encoding='utf-8') as f:
    for line in f:
        if object_concept in line:
            attributes_of_object_concept = line.split('|')[1].split('\\n')[0:-1]
            break
        
selected_concepts = list()    
with codecs.open(join(directory, full_dot), 'r', encoding='utf-8') as f:
    for line in f:
        if 'graph' in line or 'rankdir' in line or line == '}':
            out_file.write(line)
            continue
            
        if '->' in line:
            elements = line.split()
            conc1 = elements[0]
            conc2 = elements[2]
            if conc1 in selected_concepts and conc2 in selected_concepts:
                out_file.write(line)
            continue
                
        vertex_id = line.split()[0]
        concept_id = re.search('<(.*)>', line).group(1)
        if ('Object ' + str(family_number)) in line:
            selected_concepts.append(vertex_id)
            if 'E: 3920' in line: # assumption : number of families = 3920, top concept has empty intent
                elements = line.split('|')
                elements[2] = '||all families}"];\n'
                to_be_written = ''.join(elements)
                to_be_written = re.sub('\(.*\)', concept_id, to_be_written)
                out_file.write(to_be_written)
            else:
                to_be_written = line.replace('Attribute', 'Graph').replace('Object ', 'F')
                to_be_written = re.sub('\(.*\)', concept_id, to_be_written)
                out_file.write(to_be_written)
        else:
            selected = True
            for att in attributes_of_object_concept:
                if att not in line:
                    selected = False
                    break
            if selected:
                selected_concepts.append(vertex_id)
                to_be_written = line.replace('Attribute', 'Graph').replace('Object ', 'F')
                to_be_written = re.sub('\(.*\)', concept_id, to_be_written)
                out_file.write(to_be_written)
out_file.close()
os.system('dot -Tpdf ' + join(directory, out_file_name) + ' -o ' + join(directory, out_file_name).replace('.dot', '.pdf'))

In [ ]:
# filter by family, minimal
family_id = 'F01942'
simplified_dot = 'families_simplified_s100_l3.dot'
full_dot = 'families_full_s100_l3.dot'
directory = 'posets'

family_number = int(family_id.strip('F'))
out_file_name = 'posets_' + family_id + '_minimal' + '.dot'
out_file = codecs.open(join(directory, out_file_name), 'w')

object_concept = ''
with codecs.open(join(directory, simplified_dot), 'r', encoding='utf-8') as f:
    for line in f:
        if ('Object ' + str(family_number)) in line:
            object_concept = line.split()[0]
            break
            
attributes_of_object_concept = list()
with codecs.open(join(directory, full_dot), 'r', encoding='utf-8') as f:
    for line in f:
        if object_concept in line:
            attributes_of_object_concept = line.split('|')[1].split('\\n')[0:-1]
            break
        
selected_concepts = list()    
with codecs.open(join(directory, full_dot), 'r', encoding='utf-8') as f:
    for line in f:
        if 'graph' in line or 'rankdir' in line or line == '}':
            out_file.write(line)
            continue
            
        if '->' in line:
            elements = line.split()
            conc1 = elements[0]
            conc2 = elements[2]
            if conc1 in selected_concepts and conc2 in selected_concepts:
                out_file.write(line)
            continue
                
        vertex_id = line.split()[0]
        concept_id = re.search('<(.*)>', line).group(1)
        if ('Object ' + str(family_number)) in line:
            selected_concepts.append(vertex_id)
            elements = line.split('|')
            elements[0] = elements[0].replace('> (', '> ' + concept_id + '|(').replace('I', 'Graphs').replace('E', 'Families')
            #to_be_written = '|'.join(elements) + '\n'
            out_file.write(elements[0] + '}"];\n')
        else:
            selected = True
            for att in attributes_of_object_concept:
                if att not in line:
                    selected = False
                    break
            if selected:
                selected_concepts.append(vertex_id)
                elements = line.split('|')
                elements[0] = elements[0].replace('> (', '> ' + concept_id + '|(').replace('I', 'Graphs').replace('E', 'Families')
                out_file.write(elements[0] + '}"];\n')
out_file.close()
os.system('dot -Tpdf ' + join(directory, out_file_name) + ' -o ' + join(directory, out_file_name).replace('.dot', '.pdf'))